In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [ ]:

n = 1
while True:
    url= 'https://www.fondecyt.gob.pe/convocatorias?orders[xf26]=xf26&issearch=1'
    driver = webdriver.Firefox()
    driver.get(url)
    print('//li/a[contains(text(),'+ '"'+str(n)+'"'+']')
    boton_boton_pag = driver.find_element_by_xpath('//li/a[contains(text(),'+ '"'+str(n)+'"'+')]')
    boton_boton_pag.click()
    driver.quit()
    n+=1
#url2 = r.get_attribute('href')




In [ ]:


links_proyectos = []
estados_proyectos = []
fechas_proyectos = []
titulos_proyectos = []
n=1
while True:
    
    if n > 18:
        break
    print(n)
    url= 'https://www.fondecyt.gob.pe/convocatorias?orders[xf26]=xf26&issearch=1'
    driver = webdriver.Firefox()
    driver.get(url)
    
    try:
        boton_boton_pag = driver.find_element_by_xpath('//li/a[contains(text(),'+ '"'+str(n)+'"'+')]')
        boton_boton_pag.click()
    except:
        try:
            num = 10
            boton_boton_pag = driver.find_element_by_xpath('//li/a[contains(text(),'+ '"'+str(num)+'"'+')]')
            boton_boton_pag.click()
            sleep(2)
            boton_boton_pag = driver.find_element_by_xpath('//li/a[contains(text(),'+ '"'+str(n)+'"'+')]')
            boton_boton_pag.click()
        except:
            num = 14
            boton_boton_pag = driver.find_element_by_xpath('//li/a[contains(text(),'+ '"'+str(num)+'"'+')]')
            boton_boton_pag.click()
            sleep(2)
            boton_boton_pag = driver.find_element_by_xpath('//li/a[contains(text(),'+ '"'+str(n)+'"'+')]')
            boton_boton_pag.click()

    sleep(2)

    urls = driver.find_elements_by_xpath('//a[@class="entry-link"]')
    estados = driver.find_elements_by_xpath('//ul[@class="catItemTags"]')
    fechas = driver.find_elements_by_xpath('//span[@class="catItemDateCreated"]')
    titulos = driver.find_elements_by_xpath('//h3[@class="catItemTitle"]')

    for i in urls: 
        links_proyectos.append(i.get_attribute('href'))

    for i in estados: 
        estados_proyectos.append(i.text)

    for i in fechas: 
        fechas_proyectos.append(i.text)

    for i in titulos: 
        titulos_proyectos.append(i.text)
    
    driver.quit()
    n+=1

peru = pd.DataFrame()

peru['Título'] = titulos_proyectos
peru['Estado'] =estados_proyectos
peru['Fecha Cierre'] = fechas_proyectos
peru['Link']=links_proyectos


In [ ]:
peru


In [ ]:
peru2 = peru[peru['Link'].str.contains('beca') == False].reset_index(drop=True)
peru3 = peru2[peru2['Link'].str.contains('tesis') == False].reset_index(drop=True)

In [ ]:
descripciones_proyectos = []
pdfs_proyectos = []
n=0
for link in peru3['Link']:
    print(n)
    print(link)
    n+=1

    pdf = ''
    descripcion = ''
    
    url = link
    driver = webdriver.Firefox()
    driver.get(url)
    # Descripción
    try:
        boton_descripcion = driver.find_element_by_xpath('//li/a[contains(text(),"Infórmate")]')
        boton_descripcion.click()
        sleep(random.randrange(2, 3))
        descripcion = driver.find_element_by_xpath('//div[@id="myTabContent"]').text
    except:
        descripcion = ''
    # Descripcón
    try:
        boton_pdf = driver.find_element_by_xpath('//li/a[contains(text(),"Bases")]')
        boton_pdf.click()
        sleep(random.randrange(2, 3))
        pdfs = driver.find_elements_by_xpath('//div[@id="bases"]//a')
    
        pdf = ''
        for i in pdfs:
            pdf = pdf + i.get_attribute('href') + ', '
    
    except:
        pdf = ''

    
    driver.quit()
    
        
    pdfs_proyectos.append(pdf.strip(', '))
    descripciones_proyectos.append(descripcion)

    sleep(random.randrange(2, 3))

peru3['Descripción'] = descripciones_proyectos
peru3['Pdfs'] = pdfs_proyectos

peru3.to_excel('Peru.xlsx')